# Reconstruction of Stimulus Spaces from Neural Activation Sequences

This notebook contains the relevant and referenced code for Jerome Roehm's PhD Dissertation: Reconstruction of Stimulus Spaces from Neural Activation Sequences and Anti-Geometric Persistence. University of Delaware, Spring 2023.

#### Imports and background

In [2]:
#imports
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math
import scipy as sp
import itertools
import pandas as pd
from matplotlib import collections  as mc
import dionysus as d
from matplotlib.text import TextPath
import matplotlib.patches as patches
import time
from itertools import chain, combinations, product
import operator
import copy
import random
from scipy.linalg import eigh
from scipy.stats import gaussian_kde
import seaborn as sns
from sklearn import svm

#background

#distinct colors for illustrations/labeling
color_list=['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#42d4f4', '#f032e6',
            '#bfef45', '#fabebe', '#469990', '#e6beff', '#9A6324', '#fffac8', '#800000', '#aaffc3',
            '#808000', '#ffd8b1', '#e60075', '#a9a9a9']

#list of alphabet for labeling purposes
alphabet=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
          'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

## Place cell data as sequences

This section contains code for analyzing and simulating place cell data in the form of discrete sequences.

### Sequences to coactivity matrix

Methods for generating a coactivity matrix from a list of sequences.

In [3]:
#helper functions

#eliminates repeated terms in the sequence example: [3,2,3,3,3,4] becomes [3,2,3,4]
def elim_rep(data):
    seqs = copy.deepcopy(data)
    for seq in seqs:
        delete_us=[]
        for i in range(1,len(seq)):
            if seq[i] == seq[i-1]:
                delete_us.append(i)
        for index in sorted(delete_us, reverse = True):
            del seq[index]
    return seqs

#eliminates sequences of length 1
def elim_singles(data):
    final = []
    for seq in data:
        if len(seq) > 1:
            final.append(seq)
    return final

In [4]:
#main function

#data given in list of sequences, output is matrix (or weighted graph) indexed by sorted elements in sequence
#with (i,j) entry equal to number of times i appears adjacent to j in sequence set (weight of edge in graph)
#method of adj_cor is slightly different.
def seqs_to_cor(data, rtn = 'mtx', method = 'adj_cor', almost_adj_weight = 0.2, normalize=True): 
    seqs = elim_rep(data)
    
    nodes=set()
    for seq in seqs:
        for elmt in seq:
            nodes.add(elmt)
    nodes=list(nodes)
    nodes.sort()
    num_nodes = len(nodes)
    
    cor_mtx = np.zeros((num_nodes,num_nodes))
    
    if method == 'total_adj':
        for seq in seqs:
            if len(seq) >= 2:
                for i in range(len(seq)-1):
                    cor_mtx[nodes.index(seq[i]),nodes.index(seq[i+1])] += 1
                    cor_mtx[nodes.index(seq[i+1]),nodes.index(seq[i])] += 1
    elif method == 'adj_cor':
        for i in range(len(nodes)):
            for j in range(i+1,len(nodes)):
                total_app = 0
                total_tog = 0
                for seq in seqs:
                    if len(seq) >= 2:
                        for k in range(len(seq)):
                            if seq[k] in set([nodes[i], nodes[j]]):
                                if seq[k] == nodes[i]:
                                    other = nodes[j]
                                else:
                                    other = nodes[i]
                                total_app += 1
                                if k > 0 and seq[k-1] == other:
                                    total_tog += 1
                                elif k > 1 and seq[k-2] == other:
                                    total_tog += almost_adj_weight
                                elif k < len(seq)-1 and seq[k+1] == other:
                                    total_tog += 1
                                elif k < len(seq) - 2 and seq[k+2] == other:
                                    total_tog += almost_adj_weight
                if total_app>0:
                    cor_mtx[i,j] = total_tog / total_app
        cor_mtx = cor_mtx + np.transpose(cor_mtx)
    
    for i in range(num_nodes):
        cor_mtx[i,i] = 0
        
    if normalize:
        m=np.max(cor_mtx)
        if m != 0:
            cor_mtx=cor_mtx/m
        
    if rtn == 'mtx':
        return cor_mtx
    
    elif rtn == 'nodes_mtx':
        return nodes, cor_mtx
    
    elif rtn == 'graph':
        G=nx.empty_graph()
        for n in nodes:
            G.add_node(n)
        for i in range(num_nodes):
            for j in range(i):
                if cor_mtx[i,j] != 0:
                    G.add_edge(nodes[i],nodes[j], weight = cor_mtx[i,j])
        return G

### Discrete methods for analyzing the constructability of a coactivity matrix or graph

Discrete methods to evaluating how constructible a matrix or graph is.

In [ ]:
# helper functions

# from characterization of constructible graphs (c)(i) in the dissertation
def fixed_kiss_test_1(data,prt=False):
    G=data_to_graph(data)
    #testing single nodes
    for node in G.nodes():
        neighbors=set(G.neighbors(node))
        if len(neighbors)>=3:
            for s in combinations(neighbors,3):
                flag=0
                for p in s:
                    for q in s:
                        if flag==0:
                            if G.has_edge(p,q):
                                flag=1
                if flag==0:
                    if prt:
                        print('Node',node,'is connected to nodes',list(s),'none of which are connected to each other')
                    return False
    
    #testing cliques
    for clq in nx.find_cliques(G):
        clq_neighbors=set([])
        for n in clq:
            clq_neighbors=clq_neighbors.union(set(list(G.neighbors(n))))
        for n in clq:
            if n in clq_neighbors:
                clq_neighbors.remove(n)
        if len(clq_neighbors)>=3:
            for s in combinations(clq_neighbors,3):
                flag=0
                for p in s:
                    for q in s:
                        if flag==0:
                            if G.has_edge(p,q):
                                flag=1
                if flag==0:
                    if prt:
                        print('Clique',list(clq),'is connected to nodes',list(s),'none of which are connected to each other')
                    return False
    return True

# reduces data in the form of sequences, adjacency matrix, or *graph* (mainly for node naming)
def data_to_graph(data):
    if type(data)==list:
        adj_mtx,node_labels=reduce_seqs_smart(data)
        G=nx.from_numpy_matrix(adj_mtx)
    elif type(data)==np.ndarray:
        adj_mtx=data.copy()
        node_labels=list(range(adj_mtx.shape[0]))
        G=nx.from_numpy_matrix(adj_mtx)
    elif type(data)==type(nx.complete_graph(1)):
        return data
    labels={}
    for i in range(len(node_labels)):
        labels[i]=node_labels[i]
    G=nx.relabel_nodes(G,labels)
    return G

#(mainly for node naming)
def reduce_seqs_smart(seqs):
    nodes_set=set()
    for seq in seqs:
        for n in seq:
            nodes_set.add(n)
    nodes=list(nodes_set)
    nodes.sort()
    num_nodes=len(nodes)
    adj_mtx=np.zeros((num_nodes,num_nodes))
    for seq in seqs:
        for i in range(len(seq)-1):
            adj_mtx[nodes.index(seq[i]),nodes.index(seq[i+1])]=1
    adj_mtx=adj_mtx+adj_mtx.T
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i==j:
                adj_mtx[i,j]=0
            elif adj_mtx[i,j] != 0:
                adj_mtx[i,j]=1
    return adj_mtx, nodes

#helps with finding lowest cost alteration, this is a well known NP hard problem
def subset_sum(numbers, pairing, target, partial=[], code_partial=[], sols=False):
    s = sum(partial)
    
    if sols == False:
        sols = []

    # check if the partial sum is equals to target
    if s == target: 
        sols.append(code_partial)
        
    if s >= target:
        return  # if we reach the number, stop

    for i in range(len(numbers)):
        n = numbers[i]
        code = pairing[i]
        subset_sum(numbers[i+1:], pairing[i+1:], target, partial + [n],code_partial + [code], sols=sols)

    return sols



In [5]:
#main functions

#test for passing chordal [constructability characterization (c)(ii)] and kiss_test [(c)(i)]
def is_constr(data):
    G_0=data_to_graph(data)
    if fixed_kiss_test_1(G_0) == False:
        return False
    if nx.is_chordal(G_0) == False:
        return False
    return True

# Assessing how close data is to constructible using combinatorial methods to "fix" the graph.
def fix_score(data, percentile=10, thresh_with_mean = False, rtn='both', normalize = True,
              fast=True, method = 'adj_cor', almost_adj_weight = 0.2, constr_test='theory',
              time_limit = 10**5, fast_binary=False):
    start=time.time()
    if type(data) == list:
        G=seqs_to_cor(data, rtn='graph', method = method, almost_adj_weight = almost_adj_weight)
    elif type(data) == np.ndarray:
        G=nx.from_numpy_matrix(data)
    elif type(data) == type(nx.complete_graph(1)):
        G=data.copy()
        
    num_nodes = G.number_of_nodes()
    num_pos_edges = num_nodes * (num_nodes-1) / 2
    
    # if graph is unweighted, assign unit weights
    if nx.is_weighted(G) == False:
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight']=1.0
    
    node_list = list(G.nodes())
    edge_dict = nx.get_edge_attributes(G,'weight')
    edge_list = list(edge_dict.keys())
    weight_list = list(edge_dict.values())
    
    if fast:
        #going to fudge a bit here for discrete-ness and speed
        # by making edge weights integers on a scale of 0 - 23
        if len(weight_list) > 0:
            weight_list = np.rint(weight_list / np.max(weight_list) * 23)
        for e in range(len(edge_list)):
            if type(edge_list[e][0]) == np.ndarray:
                edge_list[e][0], edge_list[e][1] = tuple(edge_list[e][0]), tuple(edge_list[e][1])
            edge_dict[(edge_list[e][0],edge_list[e][1])] = weight_list[e]
    
    weights_with_non_edges = list(weight_list)
    
    while len(weights_with_non_edges) < num_pos_edges:
        weights_with_non_edges.append(0)
    
    thresh_val = round(np.percentile(np.array(weights_with_non_edges),percentile))
    
    if thresh_with_mean:
        thresh_val = (thresh_val + mean(weight_list))/2

    thresh_weights = weight_list - thresh_val
    edge_alteration_costs = abs(thresh_weights)
    
    for i in range(len(edge_alteration_costs)):
        if edge_alteration_costs[i] == 0:
            edge_alteration_costs[i] = 1
    
    TG = nx.empty_graph(0)
    for n in list(G.nodes()):
        TG.add_node(n)
    for i in range(len(edge_list)):
        if thresh_weights[i] > 0:
            TG.add_edge(edge_list[i][0],edge_list[i][1],weight=thresh_weights[i])
    
    if is_constr(TG):
        if rtn == 'both':
            return 0,[TG]
    
    best_fixes = []
    found = False
    cost = 1
    sum_edge_alt = sum(edge_alteration_costs)
    binary_max = .05 * sum_edge_alt
    
    while found == False and (time.time()-start)/60 < time_limit:
        poss_alterations = subset_sum(edge_alteration_costs,edge_list,cost)
        for alter_us in poss_alterations:
            TG1 = TG.copy()
            for edge in alter_us:
                if TG1.has_edge(edge[0],edge[1]):
                    TG1.remove_edge(edge[0],edge[1])
                else:
                    TG1.add_edge(edge[0],edge[1],weight=edge_dict[(edge[0],edge[1])])
            if constr_test == 'theory':
                if is_constr(TG1):
                    found = True
                    best_fixes.append(TG1)
                    if rtn == 'cost' or fast == True:
                        break
            elif constr_test == 'iter':
                if iter_construct_in_R1(TG1, use_as_test=True):
                    found = True
                    best_fixes.append(TG1)
                    if rtn == 'cost' or fast == True:
                        break
        if found == False:
            cost += 1
        if fast_binary and cost > binary_max:
            break
    
    if len(best_fixes)==0:
        #add small weight edges to complete the graph. minimally messes up cont optimize
        for i in range(len(node_list)):
            for j in range(i+1,len(node_list)):
                if G.has_edge(node_list[i],node_list[j]) == False:
                    G.add_edge(node_list[i],node_list[j], weight = 0.0001)
        best_fixes.append(G)
        cost = sum(edge_alteration_costs) * 0.5 #this is max cost for complete bipartite
                                                # serves as flag that the time limit was exceeded
    
    if normalize:
        cost = cost / sum(edge_alteration_costs)
    
    if rtn == 'cost':
        return cost
    if rtn == 'both':
        return cost , best_fixes

### Simulating place cell sequence data

In [6]:
# helper functions

#bump function given current position of animal and list of positions of centers of unit intervals
def bump(c_pos,i_pos):
    array = np.array(i_pos)
    d=np.clip(c_pos - i_pos,-0.4999999999,0.4999999999)
    return np.exp((4*np.square(d) / (4*np.square(d)-1)))

# helper function to compute direction
def get_dir(a,b):
    if abs((a-b)[1]) < 10**-6:
        if (a-b)[0] < 0:
            return 0
        else:
            return math.pi
    if abs((a-b)[0]) < 10**-6:
        if (a-b)[1] < 0:
            return math.pi/2
        else:
            return 3*math.pi/2
    theta = np.arctan((b[1]-a[1])/(b[0]-a[0]))
    if a[0] - b[0] > 0:
        theta = (theta + math.pi) % (2*math.pi)
    return theta

#2d bump function
def bump_2d(c_pos,disc_center):
    d = np.linalg.norm(c_pos-disc_center)
    if d >= 0.5:
        return 0
    return np.exp((4*np.square(d) / (4*np.square(d)-1)))



In [7]:
# main functions

# generate sequences in a strictly one-dimensional environment
#generates some sequences in a mildly realistic way, and shows how they were generated
def generate_seqs(pos_list,labels,start_pos='random',travel_length='random',
                  num_thoughts=10,num_seqs=10, show = False, save=False, figsize=(15,3), title='Title'):
    track_length = max(pos_list) + 0.5
    seqs = []
    paths = []
    for c in range(num_seqs):
        seq = []
        
        if start_pos != 'random':
            x_0 = start_pos * track_length
        else:
            found = False
            while found == False:
                t = np.random.normal(.5,.35)
                if 0<t and t<1:
                    found = True
                    x_0 = t * track_length
        if travel_length == 'random':
            length = np.random.normal(1,.2)
            if np.random.uniform(0,track_length) < x_0:
                length = -length
            length = length
        else:
            length = travel_length * track_length
        if num_thoughts=='random':
            num_stops = np.random.randint(5,10)
        else:
            num_stops=num_thoughts
        checkpts = np.linspace(x_0,np.clip(x_0+length,0,track_length),num_stops) #length contains direction + / -
        for x in checkpts:
            signal = []
            for i in range(len(pos_list)):
                if bump(x,pos_list[i]) > np.random.uniform(0,1):
                    signal.append(labels[i])
            random.shuffle(signal)
            for l in signal:
                seq.append(l)
        seqs.append(seq)
        paths.append(checkpts)
    
    if show:
        vals=np.zeros((1000,len(labels)))
        for j in range(1000):
            vals[j,:]=bump(np.linspace(min(min(pos_list)-0.5,0),track_length,1000)[j],np.array(pos_list))
        plt.figure(figsize=figsize, dpi=200)
        #plt.title(title,size=15)
        for i in range(len(labels)):
            if type(labels[i]) in set([int,float]):
                color_index = labels[i]%20
            else:
                color_index = i%20
            plt.plot(np.linspace(min(min(pos_list)-0.5,0),track_length,1000),vals[:,i], color = color_list[color_index%20])
            plt.plot(pos_list[i],1,marker=TextPath((-4,-2), str(labels[i])),markersize=20*np.sqrt(len(str(labels[i]))),color=color_list[color_index%20],alpha=0.5)
            plt.plot([0],[1.1],color='#FFFFFF')
        for i in range(len(paths)):
            plt.plot(paths[i],len(paths[i])*[i/10 + .05], marker = 'o', color = 'k', alpha = 0.3)
        #plt.xticks([])
        plt.yticks([])
        if save != False:
            plt.savefig(save)
        plt.show()
    if num_seqs > 0:
        return seqs
    
# function to generate and show 2d sequences
def generate_seqs_2d(pos_array, labels, start_pos='random',travel_length='random',
                      num_thoughts=10,num_seqs=10, momentum=1, show = False, save=False, figsize=(4,4)):
    track_length=np.max(pos_array[0,:]) + 0.25
    track_width=np.max(pos_array[1,:]) + 0.25
    center = .5*np.array([track_length,track_width])
    seqs=[]
    paths=[]
    for c in range(num_seqs):
        if start_pos != 'random':
            x_0 = start_pos
        else:
            x_0 = np.array([np.random.uniform(0,track_length),np.random.uniform(0,track_width)])
        if travel_length == 'random':
            length = np.random.normal(.7,.2) * (track_length*track_width)**0.5
        else:
            length = travel_length * (track_length*track_width)**0.5
        step_length = length / num_thoughts
        checkpts = np.zeros((2,num_thoughts))
        cur_pos = x_0
        checkpts[:,0] = x_0
        cur_dir = get_dir(cur_pos,center)
        for k in range(1,num_thoughts):
            found_new = False
            tries = 0
            while found_new == False:
                if k == 1:
                    new_dir = cur_dir + np.random.normal(0,1.4)
                else:
                    new_dir = cur_dir + np.random.normal(0,1.4/(momentum+10**-5))
                new_pos = cur_pos + step_length * np.array([math.cos(new_dir),math.sin(new_dir)])
                if new_pos[0]>0 and new_pos[0]<track_length and new_pos[1]>0 and new_pos[1]<track_width:
                    found_new = True
                    checkpts[:,k] = new_pos
                    cur_pos , cur_dir = new_pos , new_dir
                if tries > 10**2:
                    found_rot = False
                    rot = 0
                    while found_rot == False:
                        new_dir = cur_dir + (-1)**rot * .03 * rot
                        new_pos = cur_pos + step_length * np.array([math.cos(new_dir),math.sin(new_dir)])
                        if new_pos[0]>0 and new_pos[0]<track_length and new_pos[1]>0 and new_pos[1]<track_width:
                            found_new = True
                            found_rot = True
                            checkpts[:,k] = new_pos
                            cur_pos , cur_dir = new_pos , new_dir
                        rot += 1
                tries +=1
        paths.append(checkpts)
        
        seq = []
        for j in range(len(checkpts[0,:])):
            signal=[]
            for i in range(len(pos_array[0,:])):
                if bump_2d(checkpts[:,j],pos_array[:,i]) > np.random.uniform(0,1):
                    signal.append(labels[i])
            random.shuffle(signal)
            for l in signal:
                seq.append(l)
        seqs.append(seq)
    
    if show:
        plt.figure(figsize=figsize, dpi=150)
        plt.plot([0,track_length,track_length,0,0],[0,0,track_width,track_width,0],'k',alpha = 0.5)
        for i in range(len(paths)):
            plt.plot(paths[i][0,0],paths[i][1,0],marker = TextPath((0,0), alphabet[i]), markersize=15*np.sqrt(len(str(i))), color='k',alpha=0.3)
        for i in range(len(paths)):
            plt.plot(paths[i][0,1:],paths[i][1,1:],color='k',alpha=0.3,marker='o',markersize=2)
        for i in range(len(paths)):
            plt.plot(paths[i][0,:2],paths[i][1,:2],color='k',alpha=0.3)
        plt.axis('equal')
        
        for i in range(pos_array.shape[1]):
            plt.plot(pos_array[0,i],pos_array[1,i],marker=TextPath((0,0), str(labels[i])),linewidth=1, markeredgewidth=.25,markeredgecolor='k', color=color_list[i%20], markersize=25*np.sqrt(len(str(labels[i]))))
            for j in range(1,20):
                circle = plt.Circle((pos_array[0,i], pos_array[1,i]), 0.5/20*j, color=color_list[i%20], fill=True, alpha=.22/j+.01)
                
                ax=plt.gca()
                ax.add_patch(circle)
            circle_border = plt.Circle((pos_array[0,i], pos_array[1,i]), 0.5, color=color_list[i%20], fill=False, alpha=0.15)
            ax=plt.gca()
            #ax.add_patch(circle_border)
        plt.axis('off')
        if save != False:
            plt.savefig(save)
        plt.show()
        if num_seqs > 0:
            for i in range(num_seqs):
                print(alphabet[i%52],seqs[i])
    if num_seqs > 0:
        return seqs

In [ ]:
# method and parameter choices for generating the data for the dissertation
# will take time to reexecute
num_nodes = 8
num_seqs = 12

sessions=1000

lin_data=[]
for ses in range(sessions):
    i_pos = np.zeros(num_nodes)
    for i in range(num_nodes):
        found = False
        while found == False:
            new = np.random.uniform(0,num_nodes/3+0)
            if np.min(abs(i_pos - new)) > .08:
                found = True
                i_pos[i]=new
    labels = list(range(num_nodes))
    gen_data = generate_seqs(i_pos,labels,show = False, num_seqs = num_seqs, num_thoughts = 'random',
                            start_pos = 'random', travel_length='random', save=False)
    lin_data.append(gen_data)
    
lin_scores=[]
for gen_data in lin_data:
    lin_scores.append(fix_score(gen_data)[0])
    
box_data=[]
for ses in range(sessions):
    rand_mesh=np.zeros((2,num_nodes))
    for i in range(num_nodes):
        filled=False
        while filled==False:
            rand=np.random.uniform(0.2,1.3,2)
            found = True
            for j in range(i):
                if np.linalg.norm(rand - rand_mesh[:,j]) < .2:
                    found = False
            if found:
                rand_mesh[:,i]=rand
                filled=True
    gen_data = generate_seqs_2d(rand_mesh,list(range(num_nodes)), show=False,
                                travel_length='random',momentum=1.5,num_seqs=num_seqs)
    box_data.append(gen_data)
    
box_scores=[]
for gen_data in box_data:
    box_scores.append(fix_score(gen_data)[0])

## Place cell data as spike trains

This section contains code for analyzing and simulating place cell data in the form of spike trains.

### Spike trains to coactivity

Methods for generating a coactivity matrix from a collection of spike times for place cells

In [9]:
# helper functions

#smoothing function for vectors, also known as the 'boxcar mean'
def moving_mean(array,w):
    if w == 0:
        return array
    new=np.zeros(len(array))
    for i in range(len(array)):
        new[i]=np.mean(array[max(i-w,0):min(i+w,len(array))+1])
    return new

#scaled sliding dot product
def sliding_dot(v1,v2,max_slide,method='sum', scaled=True):
    sliding_values=[]
    for s in range(-max_slide,0):
        if scaled:
            sliding_values.append(np.dot(v1[-s:],v2[:s]) * (s/max_slide+1))
        else:
            sliding_values.append(np.dot(v1[-s:],v2[:s]))
    for s in [0]:
        sliding_values.append(np.dot(v1,v2))
    for s in range(1,max_slide+1):
        if scaled:
            sliding_values.append(np.dot(v1[:-s],v2[s:]) * (-s/max_slide+1))
        else:
            sliding_values.append(np.dot(v1[:-s],v2[s:]))
    if type(method) == int:
        return np.max(moving_mean(sliding_values,method))
    elif method == 'sum':
        return np.sum(sliding_values)
    elif method == 'max':
        return np.max(sliding_values)
    
#pick out dead spots in activity vector
def zero_runs(a):
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [10]:
# main function

# Takes data as described below
# spike_times : a list of 1D arrays with the spike times for the neurons of interst, during times of interest

#Parameters:
# w : time window/bin size
# abs_threshold : number of spikes in bin must exceed this, else returns to zero
# threshold : normalized number of spikes in bin must exceed this, else returns to zero
# mm_smooth : moving smoothing parameter for spikes in bins
# normalize : if true, normalize the activity vectors
# magnify_spikes : raise activity vectors to this power to magnify the spikes
# max_rel_time : maximum time expected between adjacent or near adjacent place fields WILL NEED TO BE ADJUSTED FOR RIPPLES
# scaled_sd : whether or not to scale the sliding dot product in proportion to the slide
# min_inactive : minimum time to classify as inactive to partition into trials / chunks
# method : method for totaling sliding dot product, sum or max or number for max of smoothed
# show : show insight into the process
# names : names for the neurons for the showing portion

def discrete_correlation(spike_times, w=0.1, abs_thresh=.7, threshold=0.2,
                         mm_smooth=3, normalize=True, magnify_spikes=1,
                         max_rel_time=3, scaled_sd=True, save=False, title=False,
                         min_inactive=5, method='sum', show=False, names=False,
                         figsize=(10,4)):
    first_spike,last_spike=10**6,0
    for n in spike_times:
        if len(n)>0:
            if np.min(n)<first_spike:
                first_spike=np.min(n)
            if np.max(n)>last_spike:
                last_spike=np.max(n)
    coactivity_windows=np.arange(first_spike-w,last_spike+w,w)
    activity_vectors=np.zeros((len(coactivity_windows)+1,len(spike_times)))
    
    for n in range(len(spike_times)):
        for spk in spike_times[n]:
            #if we wanted to consider velocity here, we could
            activity_vectors[np.digitize(spk,coactivity_windows),n]+=1
    
    #smoothing to avoid "alternating near misses"
    if mm_smooth != False:
        for i in range(len(spike_times)):
            activity_vectors[:,i]=moving_mean(activity_vectors[:,i],mm_smooth)
            
    # absolute threshold to remove one-off spikes
    if abs_thresh != False:
        for i in range(len(spike_times)):
            activity_vectors[:,i]=activity_vectors[:,i]*(activity_vectors[:,i]>abs_thresh)
            
    # normalizing to account for some neurons being more active than others
    # could get fancier later (with percentiles and such)
    if normalize:
        for i in range(len(spike_times)):
            if np.max(activity_vectors[:,i])>0:
                activity_vectors[:,i]=activity_vectors[:,i] / np.max(activity_vectors[:,i])
    
    #magnify the spikes and minimize "small activity"
    activity_vectors=activity_vectors**magnify_spikes
                
    # hard threshold to account for noise
    if threshold != False:
        for i in range(len(spike_times)):
            activity_vectors[:,i]=activity_vectors[:,i] * (activity_vectors[:,i]> threshold*np.max(activity_vectors[:,i]))

    # transposing so that it matches the matshow images intuition
    activity_vectors=activity_vectors.T
    
    #breaking into chunks so the shift does not have to be uniform for every trial, also for speed
    cols_to_delete=[(0,0)]
    zero_col_intervals=zero_runs(np.sum(activity_vectors,axis=0)>=.01)
    for i in range(zero_col_intervals.shape[0]):
        if zero_col_intervals[i,1] - zero_col_intervals[i,0] > round(1/w * min_inactive):
            cols_to_delete.append(zero_col_intervals[i,:])
    cols_to_delete.append((activity_vectors.shape[1]-1,activity_vectors.shape[1]-1))
        
    av_chunks=[]
    for i in range(len(cols_to_delete)-1):
        av_chunks.append(activity_vectors[:,cols_to_delete[i][1]:cols_to_delete[i+1][0]])
    
    #computing the extra fancy correlation matrix
    co_mtx=np.zeros((len(spike_times),len(spike_times)))
    max_slide = round(max_rel_time/w)
    for av in av_chunks:
        for i in range(len(spike_times)):
            for j in range(i+1,len(spike_times)):
                co_mtx[i,j] += sliding_dot(av[i,:],av[j,:],max_slide,method=method,scaled=scaled_sd)
                
    co_mtx=co_mtx+co_mtx.T
    
    if show:
        if names==False:
            names=list(range(len(spike_times)))
        for i in range(len(av_chunks)):
            fig = plt.figure(figsize=figsize, dpi=200)
            ax = fig.add_subplot(111)
            cax = ax.matshow(av_chunks[i],aspect=10)
            fig.colorbar(cax)
            labels=[]
            for n in names:
                labels.append(str(n))
            ax.set_yticks(np.arange(len(labels)))
            ax.set_yticklabels(labels)
            ax.set_ylabel('Neurons')
            ax.set_xlabel('Time ('+str(w)+' second bins)')
            if title != False:
                plt.title(title)
            else:
                plt.title('Chuck '+str(i))
            if save !=False:
                plt.savefig(save,dpi=200)
            plt.show()
        
        fig = plt.figure(figsize=(4,4))
        ax = fig.add_subplot(111)
        cax = ax.matshow(co_mtx)
        fig.colorbar(cax)
        plt.show()
    
    return co_mtx

### Continuous methods for analyzing the constructability of a coactivity matrix or graph
Optimization and eigen methods to evaluating how constructible a matrix or graph is.

In [11]:
# helper functions
def bp(x,y):
    d=abs(x-y)
    if d>=0.5:
        return 0
    else:
        return np.exp(4*d**2 / (4*d**2 - 1))
    
def objective_function(pos_vect,cor_mtx):
    total = 0
    for i in range(len(pos_vect)):
        for j in range(i+1,len(pos_vect)):
            total += (cor_mtx[i,j] - bp((pos_vect[i]+pos_vect[j])/2,pos_vect[i]))**2
    return total

def partial_bp_xr(pos_vect,cor_mtx,j,r):
    if abs(pos_vect[r]-pos_vect[j]) >= 1:
        return 0
    d = pos_vect[j] - pos_vect[r]
    A = np.exp(d**2 / (d**2 - 1))
    B = 2*d / (d**2 - 1)**2
    return A*B

def partial_F_xr(pos_vect,cor_mtx,r):
    total = 0
    for i in range(len(pos_vect)):
        if i != r:
            A = partial_bp_xr(pos_vect,cor_mtx,i,r)
            B = -1*cor_mtx[r,i] + bp((pos_vect[r]+pos_vect[i])/2 , pos_vect[r])
            total = total + 2*A*B
    return total

def partial_F(pos_vect,cor_mtx):
    out_vect = np.zeros(len(pos_vect))
    for r in range(len(pos_vect)):
        out_vect[r] = partial_F_xr(pos_vect,cor_mtx,r=r)
    return out_vect

def hess_F_xr_xt(pos_vect,cor_mtx,r,t):
    if abs(pos_vect[t] - pos_vect[r]) >= 1:
        return 0
    A = -1 * (partial_bp_xr(pos_vect,cor_mtx,t,r)**2)
    B = -cor_mtx[r,t] + bp((pos_vect[t]+pos_vect[r])/2,pos_vect[r])
    C = bp((pos_vect[t]+pos_vect[r])/2,pos_vect[r])
    D = (-2*(pos_vect[t] - pos_vect[r])**2-2) / ((pos_vect[t]-pos_vect[r])**2 - 1)**4
    return 2* (A+B*C*D)

def hess_F(pos_vect,cor_mtx):
    n = len(pos_vect)
    hess = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            hess[i,j] = hess_F_xr_xt(pos_vect,cor_mtx,r=j,t=i)
    return hess

In [12]:
# main functions

# The continuous optimize function:
def continuous_optimize(data,x0_dict = 'random',show=False, method = 'total_adj', node_labels=False,
                        almost_adj_weight = 0.2, fast_binary=False, save=[False,False,False],
                        data_colors=False, max_step=0, pres_style='simple'):
    
    if type(data) == np.ndarray:
        cor_mtx=data
        nodes=list(range(cor_mtx.shape[0]))
        if type(node_labels) not in [str,bool]: #ALTERED HERE
            nodes=node_labels
        num_nodes=len(nodes)
    else:
        if type(data) == list:
            G=seqs_to_cor(data, method = method, almost_adj_weight = almost_adj_weight)
        else:
            G=data

        # if graph is unweighted, assign unit weights
        if nx.is_weighted(G) == False:
            for edge in G.edges():
                G[edge[0]][edge[1]]['weight']=1.0

        nodes = list(G.nodes())
        nodes.sort()
        num_nodes = len(nodes)
        cor_mtx = nx.to_numpy_matrix(G, nodelist=nodes)
    
    if method == 'total_adj' or np.max(cor_mtx) > 1:
        cor_mtx = cor_mtx / np.max(cor_mtx) # * 0.98  #max correlation is 98%, linearly scale otherwise
    
    initial_guess = np.zeros(num_nodes)
    if x0_dict == 'random':
        for i in range(num_nodes):
            initial_guess[i] = np.random.uniform(0,1)
    else:
        for i in range(num_nodes):
            initial_guess[i] = x0_dict[nodes[i]]
    
    if fast_binary:
        options = {'initial_trust_radius':0.1,'max_trust_radius':0.2,'maxiter':10**4}
    else:
        options = {'initial_trust_radius':0.02,'max_trust_radius':0.021,'maxiter':10**5}
    if max_step != 0:
        options = {'initial_trust_radius':.99*max_step,'max_trust_radius':max_step,'maxiter':10**5}
    
    opt_results = scipy.optimize.minimize(objective_function,args = (cor_mtx),x0=initial_guess,
                            method = 'trust-ncg',jac = partial_F,
                            hess = hess_F,
                            options = options)
    opt_pos = opt_results.x
    opt_pos = opt_pos - np.min(opt_pos) + 0.5
    score = opt_results.fun
    
    if show == False:
        return opt_pos , nodes , score
    
    graph(cor_mtx, weighted = True, normalize = True, layout='circular', save=save[0],node_labels=nodes)
    print('Initial Guess:')
    generate_seqs(initial_guess,nodes,num_seqs = 0, show = True, save=save[1],data_colors=data_colors,pres_style=pres_style)
    print()
    print('Final Optimization:')
    generate_seqs(opt_pos,nodes,num_seqs = 0, show = True, save=save[2],data_colors=data_colors,pres_style=pres_style)
    print('Score:',score)
    print('Settled in local min:',opt_results.success)
    print('Num iterations:',opt_results.nit)
    
    return opt_pos , nodes , score

#method for optimization using the Fiedler eigenvalue
def eig_opt(matrix, show=False, labels=False, data_colors=True, normalize=True):
    if normalize:
        if matrix.shape[0]>0:
            mtx=matrix / np.max(matrix)
        else:
            mtx=matrix
    else:
        mtx=matrix
    L = np.diag(np.sum(mtx,axis=0)) - mtx
    w,v = eigh(L)
    sortWInds = np.argsort(w)
    fVec = v[:,sortWInds[1]]
    order=np.argsort(fVec)
    sorted_mtx=np.zeros(mtx.shape)
    for i in range(len(order)):
        for j in range(len(order)):
            sorted_mtx[i,j]=mtx[order[i],order[j]]
    
    if labels==False:
        sorted_labels=order
    else:
        sorted_labels=[]
        for i in order:
            sorted_labels.append(labels[i])
    
    if show:
        generate_seqs(np.sort(fVec)/(np.max(fVec)-np.min(fVec))*len(sorted_labels)/2.5,sorted_labels,num_seqs=0,show=True,data_colors=data_colors)
        print('Eig Opt Score:',w[sortWInds[1]])
    return w[sortWInds[1]] , fVec , sorted_labels, sorted_mtx

#seeding the continuous optimization minimizer with the result of eigen optimization
def eig_to_cont_opt(mtx, show=False, labels=False, data_colors=False, rtn='all',
                    normalize=True, show_process=False, normalize_cscore=True,
                    max_step=0,save=False, seed_with='order', start_with=False, pres_style='simple'):
    
    eig_score, start_locs, nodes, sorted_mtx = eig_opt(mtx,labels=labels)
    start_locs=np.sort(start_locs)
    
    if seed_with=='pos':
        #Scrunch together so most node pairs provide feedback
        start_locs = start_locs-np.min(start_locs)
        start_locs = start_locs / np.max(start_locs) / 1.5
        seed_dict={}
        for i in range(len(nodes)):
            seed_dict[nodes[i]]=start_locs[i]
    elif seed_with=='order':
        equi_spaced=np.arange(len(nodes))/10
        seed_dict={}
        for i in range(len(nodes)):
            if type(nodes)==np.ndarray:
                seed_dict[nodes[i]]=equi_spaced[np.where(nodes == nodes[i])[0][0]]
            else:
                seed_dict[nodes[i]]=equi_spaced[nodes.index(nodes[i])]
    
    

    c_opt_pos , nodes , c_score = continuous_optimize(sorted_mtx,x0_dict = seed_dict,
                                                      show=show_process,node_labels=nodes,
                                                      data_colors=data_colors,max_step=max_step)
    
    
    # so that matrices of differing sizes can be compared
    if normalize_cscore:
        n=len(nodes)
        comps = n*(n-1)/2
        c_score = c_score / comps
        
    #standardize so that the endpt with smaller value is first, and arrays are in order
    c_sorted_nodes=np.array(nodes)[np.argsort(c_opt_pos)]
    c_opt_pos=np.sort(c_opt_pos)
    if start_with==False and c_sorted_nodes[0] > c_sorted_nodes[-1]:
        c_sorted_nodes=np.flip(c_sorted_nodes)
        c_opt_pos=-np.flip(c_opt_pos)+c_opt_pos[-1]+c_opt_pos[0]
        nodes=nodes[::-1]
    elif start_with!=False and list(c_sorted_nodes).index(start_with)>.5*len(c_sorted_nodes):
        c_sorted_nodes=np.flip(c_sorted_nodes)
        c_opt_pos=-np.flip(c_opt_pos)+c_opt_pos[-1]+c_opt_pos[0]
        nodes=nodes[::-1]
    
    if show:
        print('Eigen Order:',nodes)
        print('Final Optimization:')
        generate_seqs(c_opt_pos,c_sorted_nodes,num_seqs = 0, show = True, save=save, data_colors=data_colors,pres_style=pres_style)
        print('Eigenvalue Optimization Score:', eig_score)
        #print('Eigen Order:',nodes)
        print('Continuous Optimization Score:',c_score)
        print(list(c_sorted_nodes))
    else:
        if rtn == 'all':
            return eig_score, nodes , c_score , c_opt_pos , c_sorted_nodes
        elif rtn == 'scores':
            return eig_score , c_score

### Simulating place cell spike train data

In [13]:
# helper functions

def two_dim_bump(pf_center,pos,m,n,w):
    d=np.linalg.norm(pf_center-pos)
    if d>=w:
        return n
    else:
        return (m-n)*np.exp(-d**2 / (w**2 - d**2))+n
    
def pt_inside_poly(point,boundry_poly_vertices):
    sides=[]
    for i in range(boundry_poly_vertices.shape[0]-1):
        sides.append(np.array([boundry_poly_vertices[i,:],boundry_poly_vertices[i+1,:]]))
    sides.append(np.array([boundry_poly_vertices[-1,:],boundry_poly_vertices[0,:]]))
    avoid_angles=[]
    for side in sides:
        if side[1,0]==side[0,0]:
            avoid_angles.append(1.57)
        else:
            avoid_angles.append(np.arctan((side[1,1]-side[0,1])/(side[1,0]-side[0,0])))
    for i in range(boundry_poly_vertices.shape[0]):
        if point[0]==boundry_poly_vertices[i,0]:
            avoid_angles.append(1.57)
        else:
            avoid_angles.append(np.arctan((boundry_poly_vertices[i,1]-point[1])/(boundry_poly_vertices[i,0]-point[0])))
    avoid_angles=np.sort(np.array(avoid_angles))
    angle=avoid_angles[np.argmax(np.diff(avoid_angles))]+avoid_angles[np.argmax(np.diff(avoid_angles))+1]
    angle=angle/2
    ray=np.array([point,point+2*np.max(boundry_poly_vertices)*np.array([np.cos(angle),np.sin(angle)])])
    intersections=0
    for side in sides:
        #plt.plot(ray[:,0],ray[:,1],color='k')
        #plt.plot(side[:,0],side[:,1],color='b')
        A=np.array([[ray[0,1]-ray[1,1] , ray[1,0]-ray[0,0]], [side[0,1]-side[1,1] , side[1,0]-side[0,0]] ])
        b=np.array([(ray[1,0] - ray[0,0])*ray[0,1] - (ray[1,1]-ray[0,1])*ray[0,0] , (side[1,0] - side[0,0])*side[0,1] - (side[1,1]-side[0,1])*side[0,0]])
        x=np.linalg.solve(A,b)
        #c='k'
        if np.min(ray[:,0])<x[0]<np.max(ray[:,0]) and np.min(ray[:,1])<x[1]<np.max(ray[:,1]):
            #print('ray good')
            #print(side,x)
            if np.min(side[:,0])-10**-6<x[0]<np.max(side[:,0])+10**-6 and np.min(side[:,1])-10**-6<x[1]<np.max(side[:,1])+10**-6:
                #print('and side good')
                intersections+=1
                #c='r'
        #plt.scatter(x[0],x[1],color=c)
        #plt.show()
    if intersections%2 == 0:
        return False
    else:
        return True
    
def crosses_boundry(old_pt,new_pt,boundry_poly_vertices):
    sides=[]
    for i in range(boundry_poly_vertices.shape[0]-1):
        sides.append(np.array([boundry_poly_vertices[i,:],boundry_poly_vertices[i+1,:]]))
    sides.append(np.array([boundry_poly_vertices[-1,:],boundry_poly_vertices[0,:]]))
    ray=np.array([old_pt,new_pt])
    for side in sides:
        A=np.array([[ray[0,1]-ray[1,1] , ray[1,0]-ray[0,0]], [side[0,1]-side[1,1] , side[1,0]-side[0,0]] ])
        b=np.array([(ray[1,0] - ray[0,0])*ray[0,1] - (ray[1,1]-ray[0,1])*ray[0,0] , (side[1,0] - side[0,0])*side[0,1] - (side[1,1]-side[0,1])*side[0,0]])
        x=np.linalg.solve(A,b)
        if np.min(ray[:,0])-10**-6<x[0]<np.max(ray[:,0])+10**-6 and np.min(ray[:,1])-10**-6<x[1]<np.max(ray[:,1])+10**-6:
            if np.min(side[:,0])-10**-6<x[0]<np.max(side[:,0])+10**-6 and np.min(side[:,1])-10**-6<x[1]<np.max(side[:,1])+10**-6:
                return True
    return False

In [16]:
# main function

def generate_2d_environment_spikes(pf_centers,pf_max_rates=30,pf_rads=1,labels=False,
                                   boundry_poly_vertices=False, hole_vertices=[],
                                   start_pos='random',travel_length='random', num_runs=10, momentum=1,
                                   show=False, noise=1, run_speed=.8, figsize='auto',save=False):
    if labels==False:
        labels=np.arange(max(pf_centers.shape))
    if type(pf_rads) in [int,float]:
        pf_rads=pf_rads*np.ones(len(labels))
    x_min , x_max = np.min(pf_centers[:,0])-np.max(pf_rads) , np.max(pf_centers[:,0])+np.max(pf_rads)
    y_min , y_max = np.min(pf_centers[:,1])-np.max(pf_rads) , np.max(pf_centers[:,1])+np.max(pf_rads)
    if type(boundry_poly_vertices)==bool:
        boundry_poly_vertices=np.array([[x_min,y_min],[x_min,y_max],[x_max,y_max],[x_max,y_min]])
    if type(noise) in [int,float]:
        neuron_noises=noise*np.ones(len(labels))
    else:
        neuron_noises=noise
    if type(pf_max_rates) in [int,float]:
        pf_max_rates=pf_max_rates*np.ones(len(labels))
    if type(pf_rads) in [int,float]:
        pf_rads=pf_rads*np.ones(len(labels))
        
    paths=[]
    max_run_time=0
    for c in range(num_runs):
        if type(travel_length)==str:
            run_time=np.random.randint(15,25)
        else:
            if type(travel_length) in [float,int]:
                run_time=round(travel_length)
            else:
                run_time=round(travel_length[c])
        if run_time>max_run_time:
            max_run_time=run_time
        if type(start_pos)==str:
            found=False
            while found==False:
                pos=np.array([np.random.uniform(np.min(boundry_poly_vertices[:,0]),np.max(boundry_poly_vertices[:,0]))
                              ,np.random.uniform(np.min(boundry_poly_vertices[:,1]),np.max(boundry_poly_vertices[:,1]))])
                outside_holes=True
                for hole in hole_vertices:
                    if outside_holes:
                        if pt_inside_poly(pos,hole):
                            outside_holes=False
                if pt_inside_poly(pos,boundry_poly_vertices) and outside_holes:
                    found=True
                    cur_pos=pos
        else:
            cur_pos=start_pos
        
        #start with course 1 second time scale to construct position as function of time
        coarse_positions=np.zeros((run_time,2))
        coarse_positions[0,:]=cur_pos
        cur_dir= np.random.normal(get_dir(cur_pos,.5*np.array([x_min+x_max,y_min+y_max])),2)
        for t in range(1,run_time):
            found_new = False
            tries = 0
            while found_new == False:
                step_length=np.abs(np.random.normal(run_speed,.15))
                new_dir = cur_dir + np.random.normal(0,1.4/(momentum+10**-5))
                new_pos = cur_pos + step_length * np.array([np.cos(new_dir),np.sin(new_dir)])
                outside_holes=True
                for hole in hole_vertices:
                    if outside_holes:
                        if crosses_boundry(coarse_positions[t-1,:],new_pos,hole)==True:
                            outside_holes=False
                if crosses_boundry(coarse_positions[t-1,:],new_pos,boundry_poly_vertices)==False and outside_holes:
                    found_new = True
                    coarse_positions[t,:]=new_pos
                    cur_pos , cur_dir = new_pos , new_dir
                elif tries > 100:
                    found_rot = False
                    rot = 0
                    while found_rot == False:
                        new_dir = cur_dir + (-1)**rot * .05 * rot
                        new_pos = cur_pos + step_length * np.array([np.cos(new_dir),np.sin(new_dir)])
                        outside_holes=True
                        for hole in hole_vertices:
                            if outside_holes:
                                if crosses_boundry(coarse_positions[t-1,:],new_pos,hole)==True:
                                    outside_holes=False
                        if crosses_boundry(coarse_positions[t-1,:],new_pos,boundry_poly_vertices)==False and outside_holes:
                            found_new = True
                            found_rot = True
                            coarse_positions[t,:]=new_pos
                            cur_pos , cur_dir = new_pos , new_dir
                        rot+=1
                tries+=1
            
        s=10
        #now make positions for every 1/s seconds and add some granular noise
        fine_positions=np.zeros((coarse_positions.shape[0]*s,2))
        for i in range(coarse_positions.shape[0]):
            fine_positions[s*i,:]=coarse_positions[i,:]
        #linear interpolation
        for i in range(coarse_positions.shape[0]-1):
            fine_positions[s*i:s*i+s,0]=np.linspace(fine_positions[s*i,0],fine_positions[s*i+s,0],s)
            fine_positions[s*i:s*i+s,1]=np.linspace(fine_positions[s*i,1],fine_positions[s*i+s,1],s)
        fine_positions=fine_positions[:-(s-1),:]
        #adding noise
        fine_positions+=np.random.normal(0,.01,fine_positions.shape)
        #smoothing
        fine_positions[:,0]=moving_mean(fine_positions[:,0],2)
        fine_positions[:,1]=moving_mean(fine_positions[:,1],2)
        paths.append(fine_positions)
        
    #now for some spike times given the positions
    spike_times=[ [] for _ in range(len(labels)) ]
    spike_pos=[ [] for _ in range(len(labels)) ]
    start_time_gap=100
    if max_run_time>start_time_gap:
        start_time_gap=round(max_run_time)+20
    start_time=-start_time_gap
    for i in range(len(labels)):
        m,n,w,c=pf_max_rates[i],neuron_noises[i],pf_rads[i],pf_centers[i,:]
        for p in range(len(paths)):
            start_time=p*start_time_gap
            for t in range(paths[p].shape[0]):
                timestamp=start_time+(t+1)/s
                expected_fires=.5/s * two_dim_bump(c,paths[p][t,:],m,n,w) #????
                if expected_fires<1:
                    if expected_fires>np.random.uniform(0,1):
                        expected_fires=1
                    else:
                        expected_fires=0
                expected_fires=round(abs(expected_fires*np.random.normal(1,neuron_noises[i])))
                for k in range(expected_fires):
                    spike_times[i].append(np.random.uniform(timestamp-1/s,timestamp+1/s))
                    spike_pos[i].append(paths[p][t,:]+np.random.normal(0,.02,2))
        spike_times[i].sort()
        spike_times[i]=np.array(spike_times[i])
        spike_pos[i]=np.array(spike_pos[i])
    
        
    if show:
        if type(figsize) != str:
            plt.figure(figsize=figsize, dpi=150)
        else:
            plt.figure(figsize=(10,10*np.max(boundry_poly_vertices[:,1])/np.max(boundry_poly_vertices[:,0])))
        plt.plot(boundry_poly_vertices[:,0],boundry_poly_vertices[:,1],color='k',alpha=0.5)
        plt.plot(boundry_poly_vertices[[-1,0],0],boundry_poly_vertices[[-1,0],1],color='k',alpha=0.5)
        for hole in hole_vertices:
            plt.plot(hole[:,0],hole[:,1],color='k',alpha=0.5)
            plt.plot(hole[[-1,0],0],hole[[-1,0],1],color='k',alpha=0.5)
        
        for i in range(len(paths)):
            plt.plot(paths[i][:,0],paths[i][:,1],color='k',alpha=0.5)
        if num_runs>0:
            for i in range(len(labels)):
                plt.scatter(spike_pos[i][:,0],spike_pos[i][:,1],color=color_list[labels[i]%20],label=labels[i],alpha=0.5)
        for i in range(len(paths)):
            plt.plot(paths[i][0,0],paths[i][0,1],marker = TextPath((0,0), alphabet[i]), markersize=20*np.sqrt(len(str(i))), color='k')
        plt.axis('equal')
        for i in range(len(labels)):
            plt.plot(pf_centers[i,0],pf_centers[i,1],marker=TextPath((0,0), str(labels[i])),linewidth=1, markeredgewidth=.4,markeredgecolor='k', color=color_list[labels[i]%20], markersize=25*np.sqrt(len(str(labels[i]))))
            for j in range(1,20):
                circle = plt.Circle((pf_centers[i,0],pf_centers[i,1]), 1/20*j, color=color_list[labels[i]%20], fill=True, alpha=(.22/j+.01)*pf_max_rates[i]/np.mean(pf_max_rates))
                ax=plt.gca()
                ax.add_patch(circle)
            circle_border = plt.Circle((pf_centers[i,0],pf_centers[i,1]), 1, color=color_list[labels[i]%20], fill=False, alpha=0.15)
            ax=plt.gca()
        plt.axis('off')
        if save != False:
            plt.savefig(save,dpi=150)
        plt.show()
        
    return spike_times

In [19]:
# Environments used in dissertation
# Defining the linear track, Y, X, circular track, 
# two dimensional box, holey octagon environments, as well as just noise.

def set_up_environments(n=21):

    # linear track
    lin_boundry=np.array([[0,0],[.5,0],[.5,1.1*n],[0,1.1*n]])
    lin_pf=np.ones((n,2))*.25
    lin_pf[:,1]=np.linspace(.4,1.1*n-.4,n)

    # box environment
    # normal square lattice
    if n != 21:
        h=round(np.floor(n**.5))
        w=round(n/h)
        box_boundry=np.array([[0,0],[1.1*w,0],[1.1*w,1.1*h],[0,1.1*h]])
        xs = np.linspace(0.4, 1.1*w-.4, w)
        ys = np.linspace(0.4, 1.1*h-.4, h)
        box_pf=[]
        for i in range(w):
            for j in range(h):
                box_pf.append([xs[i],ys[j]])
        box_pf=np.array(box_pf)
    #special case for dissertation, triangular latice
    else:
        h=3
        s3=np.sqrt(3)
        w=5*s3/2
        x1s = np.array([0,s3,2*s3])
        y1s = np.array([0,1,2,3])
        box_pf=[]
        for i in range(3):
            for j in range(4):
                box_pf.append([x1s[i],y1s[j]])
                if j<3:
                    box_pf.append([x1s[i]+s3/2,y1s[j]+0.5])
        box_pf=np.array(box_pf)
        box_boundry=np.array([[-.3,-.3],[2.5*s3+.3,-.3],[2.5*s3+.3,3.3],[-.3,3.3]])
        scale=1.2
        box_pf*=scale
        box_boundry*=scale
        

    # long loop
    k=10
    r=(n*1.1)/2/np.pi
    inner=r*np.array([np.cos(np.linspace(0,2*np.pi,k+1)),np.sin(np.linspace(0,2*np.pi,k+1))]).T
    outer=(r+.5)/r*inner
    loop_pf=r*np.array([np.cos(np.linspace(0,(n-1)/n*2*np.pi,n)),np.sin(np.linspace(0,(n-1)/n*2*np.pi,n))]).T

    # Y shape
    leg=round(n/3)
    s=1.1*leg
    q3=3**.5
    y_boundry=np.array([[-.25,0],[-.25,s],[-.25-s*q3/2,s+s/2],[-.25-s*q3/2+.25,s+s/2+q3/4],[0,s+q3/4],
                                    [.25+s*q3/2-.25,s+s/2+q3/4],[.25+s*q3/2,s+s/2],[.25,s],[.25,0]])
    y_pf=np.zeros((3*leg,2))
    y_pf[0:leg,1]=np.linspace(.3,s-.5,leg)
    y_pf[leg:2*leg,0]=np.linspace(-s*q3/2+.2,-.4,leg)
    y_pf[leg:2*leg,1]=np.linspace(s+s/2,s+.4,leg)
    y_pf[2*leg:,0]=np.linspace(s*q3/2-.2,.4,leg)
    y_pf[2*leg:,1]=np.linspace(s+s/2,s+.4,leg)

    #Holey Octagon
    ho_boundry=np.array([[0,0],[-1,1],[-1,2],[0,3],[3,3],[4,2],[4,1],[3,0]])*2
    ho_hole=[np.array([[0,1],[1.5,1],[.75,2.5]])*1.1+.3,np.array([[4,1.5],[5.6,1.5],[5.6,3],[4,3]]),
                   np.array([[1.7,5],[3.6,5.2],[3,4]])]
    ho_boundry=ho_boundry*.8
    for i in range(len(ho_hole)):
        ho_hole[i]=ho_hole[i]*.8

    ho_pf=[]
    for i in np.arange(-1.2,7,1.2):
        for j in np.arange(0.5,5,1.2):
            if pt_inside_poly(np.array([i,j]),ho_boundry):
                outside_holes=True
                for hole in ho_hole:
                    if outside_holes:
                        if pt_inside_poly(np.array([i,j]),hole):
                            outside_holes=False
                if outside_holes:
                    ho_pf.append([i,j])
    ho_pf=np.array(ho_pf)

    #Random (place fields outside environment)
    rand_boundry=np.array([[0,0],[1000,0],[1000,1000],[0,1000]])
    rand_pf=np.ones((n,2))*-10
    
    # cross shape
    m=np.max([round((n+1)/4),2])
    cross_boundry=np.array([[0,0],[0,m],[-m,m],[-m,m+.5],[0,m+.5],[0,2*m+.5],
                                [.5,2*m+.5],[.5,m+.5],[m+.5,m+.5],[m+.5,m],[.5,m],
                                [.5,0]])
    cross_boundry[:,0]+=-.25*np.ones(cross_boundry.shape[0])
    cross_pf=np.zeros((4*m-3,2))
    cross_pf[0:2*m-1,1]=np.linspace(.3,2*m+.2,2*m-1)
    cross_pf[2*m-1:,1]=m+.25
    cross_pf[2*m-1:3*m-2,0]=np.linspace(-m+.3,-1,m-1)
    cross_pf[3*m-2:,0]=np.linspace(m-.3,1,m-1)
    
    str_list=['lin','box','loop','y','ho','rand','cross']
    
    bdry={}
    bdry['lin'],bdry['box'],bdry['loop'],bdry['rand']=lin_boundry,box_boundry,outer,rand_boundry
    bdry['y'],bdry['ho'],bdry['cross']=y_boundry,ho_boundry,cross_boundry
    
    holes={}
    holes['lin'],holes['box'],holes['loop'],holes['rand']=[],[],[inner],[]
    holes['y'],holes['ho'],holes['cross']=[],ho_hole,[]
    
    pfs={}
    pfs['lin'],pfs['box'],pfs['loop'],pfs['rand']=lin_pf,box_pf,loop_pf,rand_pf
    pfs['y'],pfs['ho'],pfs['cross']=y_pf,ho_pf,cross_pf
    
    return str_list,bdry,holes,pfs

In [ ]:
# method and parameter choices for generating the data for the dissertation
# will take time to reexecute

#session params
sessions=1000
runs_per_ses=20

#navigation params
momentum=2
run_speed=.8
noise=1

all_env=set_up_environments(n=21)

for s in ['lin','box','loop','y','ho','rand','cross']:
    for ses in range(sessions):
        spike_times=generate_2d_environment_spikes(all_env[3][s],boundry_poly_vertices=all_env[1][s],
                                 hole_vertices=all_env[2][s], momentum=momentum, run_speed=run_speed,
                                 num_runs=runs_per_ses,show=False, noise=noise)
        co_mtx=discrete_correlation(spike_times)
        df=pd.DataFrame(co_mtx)
        
        #data then saved here for later analysis
        #df.to_csv('coactivity_csvs/'+s+'/'+s+'_'+str(ses)+'LONG.csv', index=False, header=None)